In [73]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [75]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-03 07:00:00+00:00


In [76]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [77]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-03 01:15:10,184 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 01:15:10,190 INFO: Initializing external client
2025-03-03 01:15:10,190 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 01:15:10,820 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649


In [78]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.63s) 


In [79]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,95,1.0,2025-03-03 03:00:00+00:00
1,179,1.0,2025-03-03 03:00:00+00:00
2,127,0.0,2025-03-03 03:00:00+00:00
3,90,71.0,2025-03-03 03:00:00+00:00
4,79,98.0,2025-03-03 03:00:00+00:00
...,...,...,...
748,93,1.0,2025-03-03 07:00:00+00:00
749,194,0.0,2025-03-03 07:00:00+00:00
750,36,0.0,2025-03-03 07:00:00+00:00
751,210,0.0,2025-03-03 07:00:00+00:00


In [80]:
#a.info()

In [81]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=0)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-03 01:15:16,918 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 01:15:16,928 INFO: Initializing external client
2025-03-03 01:15:16,928 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 01:15:17,525 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649
Current UTC time: 2025-03-03 06:15:16.918374+00:00
Next hour: 2025-03-03 06:00:00+00:00
Querying for date range: 2025-03-03 to 2025-03-04
Filtering for hour: 2025-03-03 06:00
Found 251 records


In [82]:
now = datetime.now(timezone.utc)

In [83]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
251,179,0.0,2025-03-03 06:00:00+00:00
252,10,1.0,2025-03-03 06:00:00+00:00
253,11,0.0,2025-03-03 06:00:00+00:00
254,40,1.0,2025-03-03 06:00:00+00:00
255,12,1.0,2025-03-03 06:00:00+00:00
...,...,...,...
497,97,2.0,2025-03-03 06:00:00+00:00
498,116,5.0,2025-03-03 06:00:00+00:00
499,15,0.0,2025-03-03 06:00:00+00:00
500,164,114.0,2025-03-03 06:00:00+00:00


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  753 non-null    int32                  
 1   predicted_demand    753 non-null    float64                
 2   pickup_hour         753 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 14.8 KB


In [85]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [86]:
dt = current_date.ceil('h')


In [87]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-03 01:15:18,754 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 01:15:18,761 INFO: Initializing external client
2025-03-03 01:15:18,762 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 01:15:19,329 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.64s) 


In [88]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.23s) 


,pickup_location_id,predicted_demand,pickup_hour


In [89]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,95,1.0,2025-03-03 03:00:00+00:00
1,179,1.0,2025-03-03 03:00:00+00:00
2,127,0.0,2025-03-03 03:00:00+00:00
3,90,71.0,2025-03-03 03:00:00+00:00
4,79,98.0,2025-03-03 03:00:00+00:00
...,...,...,...
748,93,1.0,2025-03-03 07:00:00+00:00
749,194,0.0,2025-03-03 07:00:00+00:00
750,36,0.0,2025-03-03 07:00:00+00:00
751,210,0.0,2025-03-03 07:00:00+00:00


In [90]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [91]:
predictions = fetch_next_hour_predictions()

2025-03-03 01:15:35,837 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 01:15:35,860 INFO: Initializing external client
2025-03-03 01:15:35,862 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 01:15:36,524 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649
Current UTC time: 2025-03-03 06:15:35.837610+00:00
Next hour: 2025-03-03 07:00:00+00:00
Found 251 records


In [92]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-03 01:15:37,522 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 01:15:37,529 INFO: Initializing external client
2025-03-03 01:15:37,531 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 01:15:38,173 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649


In [93]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.40s) 


In [94]:
df["pickup_hour"].max()

Timestamp('2025-03-03 07:00:00+0000', tz='Etc/UTC')

In [95]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-03 07:00:00+00:00


In [96]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
502,245,0.0,2025-03-03 07:00:00+00:00
503,87,23.0,2025-03-03 07:00:00+00:00
504,146,2.0,2025-03-03 07:00:00+00:00
505,175,0.0,2025-03-03 07:00:00+00:00
506,59,0.0,2025-03-03 07:00:00+00:00
...,...,...,...
748,93,1.0,2025-03-03 07:00:00+00:00
749,194,0.0,2025-03-03 07:00:00+00:00
750,36,0.0,2025-03-03 07:00:00+00:00
751,210,0.0,2025-03-03 07:00:00+00:00
